# Mortality

## Hospital mortality

In [ ]:
import pandas as pd
import numpy as np
import re
from glob import glob
import copy

file_names = glob('/mnt/dataset/dataset-2090348533109489664/EMR/*/PATIENTS.csv')

total_patients = {}

for file_name in file_names:
    _, _, _, _, _, hospital, DATE, _ = file_name.split('/')

    if hospital not in total_patients:
        total_patients[hospital] = pd.DataFrame()
        
    patients_df = pd.read_csv(file_name).drop_duplicates(['SUBJECT_ID'], ignore_index=True)
    
    total_patients[hospital] = pd.concat([total_patients[hospital], patients_df], ignore_index=True).drop_duplicates(['SUBJECT_ID'], ignore_index=True)

In [ ]:
total_rows = 0
total_DOD_count = 0

patient_dict = {}

for hospital, df in total_patients.items():
    patients_rows = len(df)
    total_rows += patients_rows
    DOD_count = len(df['DOD'].dropna())
    total_DOD_count += DOD_count
    ratio = DOD_count / patients_rows if patients_rows > 0 else 0
    
    print(f'{hospital}: n-{DOD_count}, ratio-{ratio}')

    
ratio = total_DOD_count / total_rows if total_rows > 0 else 0
print(f'total patients: {total_rows}')
print(f'\ntotal: n-{total_DOD_count}, ratio-{ratio}')

## ICU mortality

In [ ]:
import pandas as pd
import numpy as np
import re
from glob import glob
import copy

file_names = glob('/mnt/dataset/dataset-2090348533109489664/EMR/*/ADMISSIONS.csv')

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

icu_death = pd.DataFrame()

for file_name in file_names:
    _, _, _, _, _, hospital, DATE, _ = file_name.split('/')
        
    df = pd.read_csv(file_name).drop_duplicates(['SUBJECT_ID'], ignore_index=True).dropna(subset=['DEATHTIME'])
    
    for index, row in df.iterrows():
        dischtime = row['DISCHTIME']
        deathtime = row['DEATHTIME']
        sid = row['SUBJECT_ID']
        
        if not dischtime or not deathtime:
            continue
        try:
            dischtime = datetime.fromisoformat(dischtime)
            deathtime = datetime.fromisoformat(deathtime)
        except:
            continue

        new_row = {'hosp': hospital, 'sid': sid, 'death': False}
            
        if deathtime < dischtime:
            if sid not in icu_death[hospital]:
                new_row['death'] = True
        icu_death = pd.concat([icu_death, pd.DataFrame(new_row)], ignore_index=True)


In [ ]:
hosps = icu_death['hosp'].unique()
for hosp in hosps:
    temp = icu_death[icu_death['hosp'] == hosp]
    death_count = temp['death'].value_counts()[True]
    patient_count = len(temp)
    ratio = death_count / patient_count if patient_count > 0 else 0

    print(f'{hosp}: n-{death_count}, ratio-{ratio}')


ratio = icu_death['death'].value_counts()[True] / len(icu_death)
print(f'total patients: {len(icu_death)}')
print(f'\ntotal: n-{icu_death["death"].value_counts()[True]}, ratio-{ratio}')

In [ ]:
ratio = death_count / patients_rows if patients_rows > 0 else 0

# Insurance

In [ ]:
insurance_info = {}
insurance_total = {}

patients_dict = {}

file_names = glob('/mnt/dataset/dataset-2090348533109489664/EMR/*/ADMISSIONS.csv')

for file_name in file_names:
    _, _, _, _, _, hospital, DATE, _ = file_name.split('/')
        
    if hospital not in insurance_info:
        insurance_info[hospital] = {}
        
    df = pd.read_csv(file_name).drop_duplicates(['SUBJECT_ID'], ignore_index=True)
    
    for index, row in df.iterrows():
        sid = row['SUBJECT_ID']
        if sid in patients_dict:
            continue
        patients_dict[sid] = 0
        
        insurance = row['INSURANCE']
        if insurance not in insurance_info[hospital]:
            insurance_info[hospital][insurance] = 0
        insurance_info[hospital][insurance] += 1
        
        if insurance not in insurance_total:
            insurance_total[insurance] = 0
        insurance_total[insurance] += 1

In [ ]:
total_patients = sum(insurance_total.values())
print('total patients:', total_patients)

for key, count in insurance_total.items():
    print(f'{key}: {count} ({count/total_patients})\n')